In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sql-murder-mystery-database/sql-murder-mystery.db


In [84]:
import sqlite3
from pandasql import sqldf

import sqlite3
#con = sqlite3.connect("sql-murder-mystery.db")

#cursor = con.cursor()

#cursor.execute('select * from interview')
#print('DB Init')

In [69]:
pd.set_option('display.max_colwidth', None)

#setting up a connection to the database
con = sqlite3.connect('../input/sql-murder-mystery-database/sql-murder-mystery.db')

#running our first query, looking for data from crime scene report
crime_scene = "SELECT * FROM crime_scene_report where city='SQL City' and date=20180115 and type='murder'"

pd.read_sql_query(crime_scene, con)



,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [44]:
pd.read_sql_query("select * from person where address_street_name='Northwestern Dr' order by address_number desc limit 1",con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [48]:
pd.read_sql_query("select * from person where name like 'Annabel%' and address_street_name='Franklin Ave'",con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [51]:
pd.read_sql_query("select * from interview where person_id in (14887,16371)",con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


# **Details based on witness # 1**

*I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".*

In [53]:
pd.read_sql_query("select * from get_fit_now_member where id like '48Z%' and membership_status='gold'",con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


**Lets get more details of these persons**

In [61]:
pd.read_sql_query("select * from person where id in (select person_id from get_fit_now_member where id like '48Z%' and membership_status='gold')",con)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [56]:
pd.read_sql_query("select * from drivers_license where plate_number like '%H42W%'",con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


so the Murderer 'a man' used someone else's vehicle or wore someelse's gym bag to hide identity

# **Details based on witness 2**

*I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.*

In [65]:
pd.read_sql_query("select * from facebook_event_checkin where person_id in (16371,28819,67318)",con)

,person_id,event_id,event_name,date
0,16371,4719,The Funky Grooves Tour,20180115
1,67318,4719,The Funky Grooves Tour,20180115
2,67318,1143,SQL Symphony Concert,20171206


**so Jeremy bowers and witness were on same tour 'Funky Grooves Tour' on the day of the murder. So the witness would have recognized. That means may be the murderer is the other person Joe Germuska (28819)**

# **How did Joe get the car?**

In [68]:
pd.read_sql_query("select * from person where license_id in (select id from drivers_license where plate_number like '%H42W%')",con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


# **So Jeremy owns a car with plate # 0H42W2 (license id 423327) . Did Joe get Jeremy's car since Jeremy was out? **

In [70]:
pd.read_sql_query("select * from get_fit_now_member where membership_status='gold' and person_id in (28819,67318) ",con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [73]:
pd.read_sql_query("select * from person left join income on person.ssn=income.ssn where id in (28819,67318)",con)

,id,name,license_id,address_number,address_street_name,ssn,ssn,annual_income
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730,NaN,NaN
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,871539279.0,10500.0


In [83]:
pd.read_sql_query("select avg(annual_income) from income",con)

,avg(annual_income)
0,53257.798776


# Lets peek into other crime

In [80]:
pd.read_sql_query("select * from crime_scene_report where date<20180115",con)

,date,type,description,city
0,20170712,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hiding, no escape. In the distance, the beast\n falters, tethered to the void. The killing blow comes without\n hesitation, without mercy.",SQL City
1,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and future.",SQL City
2,20171110,robbery,"The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardians who fell at the Twilight Gap. Gifted\n to the survivors of that terrible battle, the Gjallarhorn\n is seen as a symbol of honor and survival.",SQL City
3,20180103,bribery,"Apparently, Cayde thought it necessary to expose this extremely\n rare vegetable to a Hive summoning ritual.",SQL City
4,20170101,fraud,forgotten to ask.’\n,Long Beach
...,...,...,...,...
958,20180114,fraud,"whether it would like the name: however, it only grinned a little wider.\n",Beaumont
959,20180114,bribery,\n,Lancaster
960,20180114,theft,\n,Kansas City
961,20180114,robbery,\n,Philadelphia


# Observations 
1. Joe has no income, he has no online presence
2. Jeremy bowers and a witness were on same tour 'Funky Grooves Tour' on the day of the murder. So the witness would have recognized. That means may be the murderer is the other person Joe Germuska (28819)